In [1]:
from glob import glob
import pandas as pd
import numpy as np
import csv
import os
%pylab inline
import matplotlib.pyplot as plt
from subprocess import check_output
from subprocess import Popen, PIPE
import shutil

Populating the interactive namespace from numpy and matplotlib


In [10]:
df = pd.read_csv("/home/sf522915/skyra_processing.csv")

In [36]:
writer = open("/home/sf522915/skyra_imaging_exams.csv", "w")
spreadsheet = csv.DictWriter(writer, fieldnames=["msID", "mseID", "ExamDate"])
spreadsheet.writeheader()

for _, row in df.iterrows():
    msid = row['msid']
    msid = msid.replace("ms", "")
    print(msid)
    cmd = ["ms_get_patient_imaging_exams", "--patient_id", msid, "--dcm_dates"]
    proc = Popen(cmd, stdout=PIPE)
    lines = [l.decode("utf-8").split() for l in proc.stdout.readlines()[5:]]
    
    tmp = pd.DataFrame(lines, columns=["mse", "date"])
    #print(tmp)
    tmp["msid"] = msid
    for _, row in tmp.iterrows():
        mse = row["mse"]
        date = row["date"]
        #mse = tmp["mse"]
        #mse = str(mse)
        #date = tmp["date"]
        towrite = {}
        towrite["msID"] = msid
        towrite["mseID"] = mse
        towrite["ExamDate"] = date
        spreadsheet.writerow(towrite)
    #break
writer.close()

1926
1092
0086
1610
0792
1597
2110
1123
2174
1821
1071
1470
1620
2016
1784
0823
1807
1979
1678


In [37]:
sd = pd.read_csv("/home/sf522915/skyra_imaging_exams.csv")

In [77]:
writer = open("/home/sf522915/skyra_imaging_exam.csv", "w")
spreadsheet = csv.DictWriter(writer, fieldnames=["msID", "mseID", "ExamDate","freesurfer", "nifti", "lst", "sienax"])
spreadsheet.writeheader()

for _, row in sd.iterrows():
    msid = "ms" + str(row["msID"])
    mse = row['mseID']
    mse = "mse"+ str(mse)
    date = row["ExamDate"]
    nifti_path = os.path.join("/data/henry7/PBR/subjects/"+ mse + "/nii/")
    mc_path = os.path.join("/data/henry7/PBR/subjects/"+ mse + "/mindcontrol/")
    freesurfer_path = "/data/henry6/PBR/surfaces/"+ msid + "-"+ mse
    sienax_path = "/data/pelletier1/genentech/sienax_yr89_lst_lesions/"
    
    towrite = {}
    towrite["msID"] = msid
    towrite["mseID"] = mse
    towrite["ExamDate"] = date
    
    #check if nifti exists 
    if os.path.exists(nifti_path):
        towrite["nifti"] = "yes"
    else:
        towrite["nifti"] = "no"
        
    #check if mindcontrol exists 
    if os.path.exists(mc_path):
        towrite["lst"] = "yes"
    else:
        towrite["lst"] = "no"
        
    #check if sienax exists 
    ms_zfill = msid.replace("ms","")
    ms_zfill = "ms" + ms_zfill.zfill(4)

    for sienax in os.listdir(sienax_path):
        if sienax.startswith(ms_zfill + "-" + mse):
            towrite["sienax"] = "yes"

    
    
    #check if freesurfer exists
    sw = msid + "-"+ mse
    for freesurfer in os.listdir("/data/henry6/PBR/surfaces/"):
        if freesurfer.startswith(sw):
            print(freesurfer)
            towrite["freesurfer"] = freesurfer
        #else:
            #towrite["freesurfer"]="no"
            
    spreadsheet.writerow(towrite)
writer.close()
        

ms823-mse6687-005-MPRAGE_iso_1
ms1620-mse4216-005-MPRAGE_iso_1
ms792-mse4433-003-MPRAGE_iso_1
ms86-mse4295-004-MPRAGE_iso_1
ms1123-mse4572-006-MPRAGE_iso_1
ms1678-mse3796-006-MPRAGE_iso_1
ms1470-mse4682-007-MPRAGE_iso_1
ms1597-mse4705-007-MPRAGE_iso_1
ms1092-mse4548-007-MPRAGE
ms1610-mse4947-007-MPRAGE
ms1610-mse4947-008-MPRAGE_MPR_TRA
ms1784-mse4097-007-MPRAGE
ms1678-mse4978-007-MPRAGE
ms1678-mse4978-008-MPRAGE_MPR_TRA
ms1807-mse5005-007-MPRAGE
ms1807-mse5005-008-MPRAGE_MPR_TRA
ms1821-mse4100-007-MPRAGE
ms86-mse4707-024-MPRAGE
ms792-mse2852-025-MPRAGE_64channel_p2
ms823-mse2866-030-MPRAGE_64channel_p2
ms1610-mse2878-029-MPRAGE_64channel_p2
ms1784-mse3026-030-MPRAGE_64channel_p2
ms1926-mse3061-024-MPRAGE_64channel_p2
ms1979-mse5184-002-Sag_T1
ms1979-mse3227-035-MPRAGE_64channel_p2
ms1979-mse3227-056-rescan_MPRAGE_64channel_p2
ms1678-mse3338-037-MPRAGE_64channel_p2
ms1807-mse3703-037-MPRAGE_64channel_p2
ms2110-mse4017-035-MPRAGE_64channel_p2
ms2110-mse4017-036-MPRAGE_64channel_p2_MPR_TR

In [90]:
ps = pd.read_csv("/home/sf522915/patient_PSIR.csv")

In [97]:
writer = open("/home/sf522915/psir2.csv", "w")
spreadsheet = csv.DictWriter(writer, fieldnames=["EPICID", "MSID", "Brain_MRI_Date","Spine_MRI_Date", "mse", "date"])
spreadsheet.writeheader()

for _, row in ps.iterrows():
    
    msid = row["MSID"]
    msid = msid.replace("ms","")
    msid = msid.lstrip("0")
    
    date1 = row["Brain_MRI_Date"]
    Spine_MRI_Date = row["Spine_MRI_Date"]
    EPICID = row["EPICID"]
    
    towrite = {}
    towrite["EPICID"] = EPICID
    towrite["Spine_MRI_Date"] = Spine_MRI_Date

    cmd = ["ms_get_patient_imaging_exams", "--patient_id", msid, "--dcm_dates"]
    proc = Popen(cmd, stdout=PIPE)
    lines = [l.decode("utf-8").split() for l in proc.stdout.readlines()[5:]]
    tmp = pd.DataFrame(lines, columns=["mse", "date"])
    tmp["msid"] = msid
    mse = tmp["mse"]
    date2 = tmp["date"]
    for _, row in tmp.iterrows():
        mse = row["mse"]
        date2 = row["date"]
        if str(date1) == str(date2):
            #print(msid, mse, date2, date1)

           
            print(EPICID, msid, mse, date1, date2)
            towrite["MSID"] = msid
            towrite["mse"] = mse
            towrite["Brain_MRI_Date"] = str(date1)
            towrite["date"] = date2
            spreadsheet.writerow(towrite) 
writer.close()
    

AttributeError: 'float' object has no attribute 'replace'

In [94]:
writer = open("/home/sf522915/psir.csv", "w")
spreadsheet = csv.DictWriter(writer, fieldnames=["EPICID", "MSID", "Brain_MRI_Date","Spine_MRI_Date", "mse", "date"])
spreadsheet.writeheader()

for _, row in ps.iterrows():
    
    msid = row["MSID"]
    msid = msid.replace("ms","")
    msid = msid.lstrip("0")
    
    date1 = row["Brain_MRI_Date"]
    Spine_MRI_Date = row["Spine_MRI_Date"]
    EPICID = row["EPICID"]
    
    towrite = {}
    towrite["EPICID"] = EPICID
    towrite["Spine_MRI_Date"] = Spine_MRI_Date

    cmd = ["ms_get_patient_imaging_exams", "--patient_id", msid, "--dcm_dates"]
    proc = Popen(cmd, stdout=PIPE)
    lines = [l.decode("utf-8").split() for l in proc.stdout.readlines()[5:]]
    tmp = pd.DataFrame(lines, columns=["mse", "date"])
    tmp["msid"] = msid
    mse = tmp["mse"]
    date2 = tmp["date"]
    for _, row in tmp.iterrows():
        mse = row["mse"]
        date2 = row["date"]
        print(EPICID, msid, mse, date1, date2)
        towrite["MSID"] = msid
        towrite["mse"] = mse
        towrite["Brain_MRI_Date"] = str(date1)
        towrite["date"] = date2
        spreadsheet.writerow(towrite) 
writer.close()
    

EPIC0001 494 1598 20131212 20040707
EPIC0001 494 1599 20131212 20050713
EPIC0001 494 1600 20131212 20060622
EPIC0001 494 1601 20131212 20070918
EPIC0001 494 1602 20131212 20081030
EPIC0001 494 1603 20131212 20091021
EPIC0001 494 4325 20131212 20131212
EPIC0002 493 7515 20130401 20011001
EPIC0002 493 2806 20130401 20011001
EPIC0002 493 1591 20130401 20040707
EPIC0002 493 1592 20130401 20050628
EPIC0002 493 1593 20130401 20060727
EPIC0002 493 2785 20130401 20061027
EPIC0002 493 1594 20130401 20071004
EPIC0002 493 1595 20130401 20081016
EPIC0002 493 1596 20130401 20091021
EPIC0002 493 1597 20130401 20101118
EPIC0002 493 2799 20130401 20120130
EPIC0002 493 2798 20130401 20120130
EPIC0002 493 2784 20130401 20120822
EPIC0002 493 2807 20130401 20121004
EPIC0002 493 5814 20130401 20130401
EPIC0002 493 5816 20130401 20130401
EPIC0002 493 2793 20130401 20130705
EPIC0002 493 2792 20130401 20140414
EPIC0002 493 6702 20130401 20160825
EPIC0002 493 7515 20140414 20011001
EPIC0002 493 2806 20140414 2

AttributeError: 'float' object has no attribute 'replace'

In [98]:
ms = pd.read_csv("/home/sf522915/EPICORIGINS.csv")

In [113]:
writer = open("/home/sf522915/EPICORIGINS_psir.csv", "w")
spreadsheet = csv.DictWriter(writer, fieldnames=["EPICID", "MSID", "Brain_MRI_Date","Spine_MRI_Date", "mse", "date", "psir","nifti"])
spreadsheet.writeheader()

for idx, row in ms.iterrows():
    msid = row["MSID"]
    
    date = row["Brain_MRI_Date"]
    Spine_MRI_Date = row["Spine_MRI_Date"]
    EPICID = row["EPICID"]
    
    mse = row.mse
    mse = "mse"+ str(mse)
    #print(date, mse)
    
    towrite = {}
    towrite["EPICID"] = EPICID
    towrite["Spine_MRI_Date"] = Spine_MRI_Date
    towrite["MSID"] = msid
    towrite["mse"] = mse
    towrite["date"] = date
    
    if os.path.exists("/data/henry7/PBR/subjects/"+ mse + "/nii/"):
        towrite["nifti"] = "yes"
        for psir in os.listdir("/data/henry7/PBR/subjects/"+ mse + "/nii/"):
            if "PSIR" in psir:
                psir = psir 
                #print(psir)
                towrite["psir"] = psir
    elif "nan" in mse:
        continue
    else:
        print(mse)
            

    
    spreadsheet.writerow(towrite) 
writer.close()
    


    

mse7189
mse6843
mse6936
mse6831
mse5733
mse6930
mse6834
mse7162
mse5736
mse6941
mse5831
mse6953
mse6931
mse7168
mse6975
mse7166
mse7158
mse7234
mse5775
mse5783
mse7220
mse5737
mse7159
mse7388
mse5740
mse7285
mse7287
mse5649
mse7286
mse7387
mse5773
mse5786
mse7547
mse6802
mse7405
mse5742
mse7729
mse7167
mse7563
mse7552
mse7548
mse7571
mse4837
mse5647
mse7557
mse7572
mse5792
mse5023
mse7565
mse7269
mse4260
mse7570
mse7737
mse5743
mse4839
mse7732
mse5796
mse5655
mse4849
mse5652
mse4262
mse5799
mse5653
mse7131
mse7424
mse5747
mse5803
mse5706
mse7558
mse8143
mse5819
mse4453
mse6804
mse8135
mse8149
mse8142
mse8443
mse8132
mse8136
mse5804
mse7186
mse5813
mse8468
mse7420
mse8148
mse7970
mse4438
mse7270
mse8137
mse8439
mse5753
mse5704
mse5755
mse5830
mse5820
mse6712
mse8452
msex
mse5440
mse6531
mse5882
msex
mse8453
mse5169
mse5380
mse4257
mse5778
mse5410
mse5230
mse5627
msex
mse5835
mse5389
msex
mse6783
mse6836
mse5765
mse4559
mse6947
mse6649
mse6811
mse6821
mse6823
mse6963
mse6963
msex
mse5255